In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder\
    .appName("Préparation de données")\
    .enableHiveSupport()\
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF")
spark.catalog.clearCache()
spark.sql("USE concessionnaire")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/15 09:22:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame[]

In [4]:
df_catalogue = spark.sql("SELECT * FROM catalogue;")
df_catalogue.show()

+------+-------------+---------+-----------+--------+--------+-------+--------+-------+-----------+----------+------------+
|marque|       modele|puissance|   longueur|nbplaces|nbportes|couleur|occasion|   prix|bonus_malus|rejets_co2|cout_energie|
+------+-------------+---------+-----------+--------+--------+-------+--------+-------+-----------+----------+------------+
|nissan|maxima 3.0 v6|      200|tres longue|       5|       5|  rouge|   false|30000.0|     8753.0|     200.0|       810.0|
|nissan|maxima 3.0 v6|      200|tres longue|       5|       5|  rouge|   false|30000.0|     8753.0|     200.0|       810.0|
|nissan|maxima 3.0 v6|      200|tres longue|       5|       5|  rouge|   false|30000.0|     8753.0|     200.0|       810.0|
|nissan|maxima 3.0 v6|      200|tres longue|       5|       5|  rouge|   false|30000.0|     8753.0|     200.0|       810.0|
|nissan|maxima 3.0 v6|      200|tres longue|       5|       5|  rouge|   false|30000.0|     8753.0|     200.0|       810.0|
|nissan|